In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Ruta al video
video_path = "../videos/video1.mp4"  # Cambia al video deseado

cap = cv2.VideoCapture(video_path)

frame_count = 0
prev_frame = None
num_prev_frame = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # Convertir a escala de grises
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Procesar solo en múltiplos de 30
    if frame_count % 30 == 0:
        if prev_frame is not None:
            # Calcular diferencias
            diff = cv2.absdiff(prev_frame, frame_gray)
            
            # Umbralización
            _, diff_thresh = cv2.threshold(diff, 70, 255, cv2.THRESH_BINARY)
            
            # Mostrar resultados
            plt.figure(figsize=(15, 10))
            plt.subplot(1, 3, 1)
            plt.title(f"Frame Base {num_prev_frame}")
            plt.imshow(prev_frame, cmap="gray")
            plt.axis("off")

            plt.subplot(1, 3, 2)
            plt.title(f"Frame Actual {frame_count}")
            plt.imshow(frame_gray, cmap="gray")
            plt.axis("off")

            plt.subplot(1, 3, 3)
            plt.title("Diferencia (Umbralizada)")
            plt.imshow(diff_thresh, cmap="gray")
            plt.axis("off")
            plt.show()

        # Actualizar el frame base
        prev_frame = frame_gray
        num_prev_frame = frame_count

cap.release()



In [9]:
import cv2
import numpy as np

# Ruta al video
video_path = "../videos/video1.mp4"  # Cambia al video deseado

cap = cv2.VideoCapture(video_path)

frame_count = 0
prev_frame = None

# Crear un objeto para detección de movimiento (fondo acumulado)
fgbg = cv2.createBackgroundSubtractorMOG2()

# Definir un tamaño más pequeño para la ventana
resize_width = 640  # Ancho deseado
resize_height = 360  # Alto deseado

# Reducir la resolución del video para mejorar el rendimiento
downscale_factor = 0.5  # Reducción al 50% del tamaño original
pixeles = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # Reducir la resolución globalmente
    frame_resized = cv2.resize(frame, None, fx=downscale_factor, fy=downscale_factor)
    
    # Convertir a escala de grises
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

    # Inicializar `diff_thresh` como una matriz vacía para evitar errores
    diff_thresh = np.zeros_like(frame_gray)

    # Procesar solo en múltiplos de 5 frames para reducir la carga de trabajo
    if frame_count % 30 == 0:
        if prev_frame is not None:
            # Calcular diferencia entre el frame actual y el anterior
            diff = cv2.absdiff(prev_frame, frame_gray)
            
            # Umbralización para obtener la diferencia binaria
            _, diff_thresh = cv2.threshold(diff, 90, 255, cv2.THRESH_BINARY)

            # Aplicar dilatación para ampliar las áreas detectadas (mejorar contornos)
            diff_thresh = cv2.dilate(diff_thresh, None, iterations=2)

            # Contar los píxeles cambiados (es decir, detectar el movimiento)
            non_zero_count = cv2.countNonZero(diff_thresh)

            # Si el número de píxeles cambiados es mayor que un umbral, se detecta movimiento
            if non_zero_count > 3200:
                print(f"Movimiento detectado en el frame {frame_count} Y PIXELES {non_zero_count}")
            else:
                print(f"SIN MOVIMIENTO PIXELES {non_zero_count}")
            pixeles.append(non_zero_count)

        # Actualizar el frame base
        prev_frame = frame_gray

    # Redimensionar las imágenes a un tamaño más pequeño para su visualización
    diff_resized = cv2.resize(diff_thresh, (resize_width, resize_height))

    # Mostrar el video y las diferencias
    cv2.imshow('Video Original', frame_resized)
    cv2.imshow('Diferencias Umbralizadas', diff_resized)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print(f"Pixeles en orden: {sorted(pixeles)}")


Movimiento detectado en el frame 60 Y PIXELES 24319
Movimiento detectado en el frame 90 Y PIXELES 19815
Movimiento detectado en el frame 120 Y PIXELES 33342
Movimiento detectado en el frame 150 Y PIXELES 18694
Movimiento detectado en el frame 180 Y PIXELES 25484
Movimiento detectado en el frame 210 Y PIXELES 14419
SIN MOVIMIENTO PIXELES 2212
SIN MOVIMIENTO PIXELES 240
SIN MOVIMIENTO PIXELES 2323
SIN MOVIMIENTO PIXELES 1600
SIN MOVIMIENTO PIXELES 3134
SIN MOVIMIENTO PIXELES 2226
SIN MOVIMIENTO PIXELES 2145
SIN MOVIMIENTO PIXELES 1251
SIN MOVIMIENTO PIXELES 226
Movimiento detectado en el frame 510 Y PIXELES 10121
Movimiento detectado en el frame 540 Y PIXELES 3352
Movimiento detectado en el frame 570 Y PIXELES 17490
Movimiento detectado en el frame 600 Y PIXELES 30491
Movimiento detectado en el frame 630 Y PIXELES 13850
Movimiento detectado en el frame 660 Y PIXELES 3640
Movimiento detectado en el frame 690 Y PIXELES 9682
Movimiento detectado en el frame 720 Y PIXELES 15144
Movimiento de

Codigo de funcion

In [ ]:
def frame_differencing(self, fragment_bytes)